In [2]:
from prepared_data.get_prepared_data import  get_prepared_data
import pandas as pd
import numpy as np


In [3]:
train_org = pd.read_csv('../data/train.csv')
train, league_mean_ratting_all, teams_ratting_all  = get_prepared_data(train_org, number_of_history_matches= 8,)
train

/home/andrzej/PycharmProjects/Physionet_Challenge_2021/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


correction col type..
percent of object with nan value:  42.20
remove coach columns..
percent of object with nan value:  18.64
remove object with more that  col 80 empty...
percent of object with nan value:  18.64
remove history columns
percent of object with nan value:  15.11
remove all nan data
percent of object with nan value:  15.11


,target,is_cup,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,...,home_team_history_target_7_1.0,away_team_history_target_7_-1.0,away_team_history_target_7_0.0,away_team_history_target_7_1.0,home_team_history_target_8_-1.0,home_team_history_target_8_0.0,home_team_history_target_8_1.0,away_team_history_target_8_-1.0,away_team_history_target_8_0.0,away_team_history_target_8_1.0
id,,,,,,,,,,,,,,,,,,,,,
11989265,-1,1,78,258,75,193,67,103,88,74,...,1,0,0,1,1,0,0,0,1,0
16865249,1,1,219,168,286,71,49,3145,67,76,...,0,0,0,1,0,0,1,0,0,1
16916583,1,1,94,627,93,99,93,74,168,69,...,0,1,0,0,0,1,0,0,0,1
16940078,-1,1,167,168,119,123,117,121,96,96,...,1,0,0,1,0,0,1,0,0,1
17213503,1,1,100,92,76,144,92,126,238,72,...,0,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17882059,1,1,504,1135,712,168,168,168,505,168,...,0,0,1,0,0,1,0,1,0,0
17887522,-1,1,127,167,208,72,96,127,545,1133,...,0,1,0,0,1,0,0,1,0,0
17882061,-1,1,545,1133,715,168,168,168,505,168,...,0,0,0,1,1,0,0,0,0,1


In [3]:
import  re
used = set()
columns_only_first_hist = [column for column in train.columns.values if
                           re.sub('\d', '', column) not in used and (used.add(re.sub('\d', '', column)) or True)]
print(columns_only_first_hist)

['target', 'is_cup', 'home_team_history_match_date_1', 'home_team_history_is_play_home_1', 'home_team_history_is_cup_1', 'home_team_history_goal_1', 'home_team_history_opponent_goal_1', 'home_team_history_rating_1', 'home_team_history_opponent_rating_1', 'away_team_history_match_date_1', 'away_team_history_is_play_home_1', 'away_team_history_is_cup_1', 'away_team_history_goal_1', 'away_team_history_opponent_goal_1', 'away_team_history_rating_1', 'away_team_history_opponent_rating_1', 'home_team_history_gol_difference_1', 'home_team_mean_regeneration_time', 'away_team_history_gol_difference_1', 'away_team_mean_regeneration_time', 'home_team_mean_ratting', 'away_team_mean_ratting', 'league_id_ratting', 'sum_history_targets', 'home_team_history_target_1_-1.0', 'home_team_history_target_1_0.0', 'away_team_history_target_1_-1.0', 'away_team_history_target_1_0.0']


In [4]:
test_org = pd.read_csv('../data/test.csv')
test,_,_  = get_prepared_data(test_org, 8, league_mean_ratting_all, teams_ratting_all )
test.isnull().values.any()

correction col type..
percent of object with nan value:  39.35
remove coach columns..
percent of object with nan value:  13.93
remove object with more that  col 80 empty...
percent of object with nan value:  13.93
remove history columns
percent of object with nan value:  11.15
remove all nan data
percent of object with nan value:  11.15


False

# Dodanie cech
- Sredni ranking drużyn w lidzie
- Sredni ranking ligi
- Intensywnosc druzyny, na podstawie dat rozegranych meczów
- Historyczny mecz target, na postawie bramek




In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

for data, name in zip([train], ['train']):
    X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0], test_size=0.33,
                                                        random_state=42)
    pipe = make_pipeline(StandardScaler(), LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr'))

    pipe.fit(X_train, y_train)
    results = pipe.predict(X_train)
    print(f'accuracy: {pipe.score(X_test, y_test):.04f}')
    print(name, results)
# drzewach, svm
# kolumny
# kaggle
#

accuracy: 0.4988
train [ 1  1  1 ...  1 -1  1]


# Dobór cech

In [7]:
from sklearn.feature_selection import SequentialFeatureSelector

X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0], test_size=0.33,
                                                    random_state=42)

svc = make_pipeline(StandardScaler(), LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr'))

sfs = SequentialFeatureSelector(svc, n_features_to_select=10,cv= 5, n_jobs=-1)
sfs.fit(X_train, y_train)

X_train.columns[ sfs.get_support()]

Index(['away_team_history_is_play_home_4', 'away_team_history_is_cup_6',
       'home_team_history_gol_difference_3', 'home_team_mean_ratting',
       'away_team_mean_ratting'],
      dtype='object')